In [1]:
#%pip install pandas requests folium ipywidgets matplotlib numpy

In [2]:
# Instalacja wymaganych bibliotek w Twoim nowym środowisku Python 3.12.1
# Zgodnie ze stosem technologicznym z prezentacji (Slajd 4)
%pip install pandas requests folium ipywidgets matplotlib numpy

import pandas as pd
import requests
import folium
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Konfiguracja estetyki wykresów
plt.style.use('seaborn-v0_8-whitegrid')
print("✅ Środowisko gotowe! Biblioteki załadowane.")


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ Środowisko gotowe! Biblioteki załadowane.


In [3]:
def get_solar_data(lat, lon):
    """
    Pobiera dane z Open-Meteo API dla wskazanej lokalizacji.
    Zwraca: roczna suma nasłonecznienia (kWh/m2).
    """
    try:
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": "2022-01-01",
            "end_date": "2022-12-31",
            "daily": "shortwave_radiation_sum",
            "timezone": "auto"
        }
        # Zapytanie do zewnętrznego źródła danych (Slajd 5)
        response = requests.get(url, params=params)
        data = response.json()
        
        # Przeliczenie MJ/m2 na kWh/m2 (fizyka budowli)
        daily_kwh = np.array(data['daily']['shortwave_radiation_sum']) * 0.277
        return daily_kwh.sum()
    except Exception as e:
        print(f"Błąd API: {e}")
        return 0

def calculate_roi(annual_production, system_cost, price, inflation, years=15):
    """
    Model ekonometryczny ROI uwzględniający inflację energii.
    """
    cumulative_cash = [-system_cost]
    current_price = price
    
    for _ in range(years):
        savings = annual_production * current_price
        cumulative_cash.append(cumulative_cash[-1] + savings)
        current_price *= (1 + inflation/100)
        
    return cumulative_cash

In [4]:
# --- WARSTWA GIS (Slajd 6) ---
# Domyślnie ustawiamy widok na Polskę
lat_center, lon_center = 52.0, 19.0

m = folium.Map(location=[lat_center, lon_center], zoom_start=6)

# Dodajemy instrukcję dla użytkownika
folium.Marker(
    [54.35, 18.65], 
    popup="Przykładowa lokalizacja: Gdańsk", 
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(m)

# Funkcja pozwalająca odczytać współrzędne po kliknięciu
folium.LatLngPopup().add_to(m)

print("👇 Użyj mapy, aby znaleźć współrzędne dla inwestycji (kliknij w dowolny punkt):")
display(m)

👇 Użyj mapy, aby znaleźć współrzędne dla inwestycji (kliknij w dowolny punkt):


In [ ]:
# --- INTERFEJS UŻYTKOWNIKA (RAPID PROTOTYPING - Slajd 7) ---
style = {'description_width': 'initial'}

# Widgety (pola do wpisywania)
w_lat = widgets.FloatText(value=54.39, description='Szerokość (Lat):', step=0.01, style=style)
w_lon = widgets.FloatText(value=18.57, description='Długość (Lon):', step=0.01, style=style)
w_power = widgets.FloatSlider(value=5.0, min=2.0, max=50.0, step=0.5, description='Moc (kWp):', style=style)
w_cost = widgets.FloatText(value=25000, description='Koszt (PLN):', style=style)
w_price = widgets.FloatSlider(value=0.90, min=0.50, max=2.50, step=0.01, description='Cena prądu (PLN):', style=style)
w_infl = widgets.IntSlider(value=10, min=0, max=25, description='Inflacja (%):', style=style)

btn_calc = widgets.Button(description="🚀 Uruchom Analizę", button_style='success', icon='chart-line')
out = widgets.Output()

def on_click_action(_):
    with out:
        clear_output(wait=True)
        print("⏳ Pobieranie danych satelitarnych i obliczanie modelu...")
        
        # 1. Pobranie twardych danych (Data Engineering)
        rad = get_solar_data(w_lat.value, w_lon.value)
        
        if rad == 0:
            print("❌ Błąd pobierania danych dla tej lokalizacji.")
            return

        prod = rad * w_power.value * 0.8 # Współczynnik wydajności 0.8
        
        # 2. Obliczenia finansowe
        cash_flow = calculate_roi(prod, w_cost.value, w_price.value, w_infl.value)
        years = range(len(cash_flow))
        profit = cash_flow[-1]
        
        # 3. Wizualizacja
        plt.figure(figsize=(9, 5))
        plt.plot(years, cash_flow, marker='o', linewidth=2, color='#27ae60')
        plt.axhline(0, color='red', linestyle='--', label='Próg rentowności (0 PLN)')
        plt.fill_between(years, cash_flow, 0, where=(np.array(cash_flow)>=0), color='#27ae60', alpha=0.1)
        
        plt.title(f"Analiza Rentowności (ROI) - Produkcja: {prod:.0f} kWh/rok", fontsize=12)
        plt.xlabel("Lata eksploatacji")
        plt.ylabel("Skumulowany zysk (PLN)")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()
        
        print(f"💰 Wynik finansowy po 15 latach: {profit:,.2f} PLN")
        if profit > 0:
            print("✅ Inwestycja jest opłacalna.")
        else:
            print("⚠️ Inwestycja nie zwróci się w założonym czasie.")

btn_calc.on_click(on_click_action)

# Układ panelu
panel = widgets.VBox([
    widgets.HTML("<h3>☀️ Panel Analityczny Inwestora</h3>"),
    widgets.HBox([w_lat, w_lon]),
    widgets.HBox([w_power, w_cost]),
    widgets.HBox([w_price, w_infl]),
    widgets.HTML("<br>"),
    btn_calc,
    out
])

display(panel)

In [6]:
%pip install streamlit streamlit-folium

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/c0/95/6b7873f0267973ebd55ba9cd33a690b35a116f2779901ef6185a0e21864d/streamlit-1.52.2-py3-none-any.whl.metadata
  Obtaining dependency information for streamlit-folium from https://files.pythonhosted.org/packages/72/35/d3cdab8cff94971714f866181abb1aa84ad976f6e7b6218a0499197465e4/streamlit_folium-0.25.3-py3-none-any.whl.metadata
  Obtaining dependency information for altair!=5.4.0,!=5.4.1,<7,>=4.0 from https://files.pythonhosted.org/packages/db/33/ef2f2409450ef6daa61459d5de5c08128e7d3edb773fefd0a324d1310238/altair-6.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.5.0 from https://files.pythonhosted.org/packages/10/cb/f2ad4230dc2eb1a74edf38f1a38b9b52277f75bef262d8908e60d957e13c/blinker-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<7,>=4.0 from https://files.pythonhosted.org/packages/2c/fc/1d7b80d0eb7b714984ce40efc78859c022cd930